In [1]:
import os

from config import SETTINGS

import seabeepy as sb

# Manually publish NodeODM mosaics from flight folders

Useful for tidying up when there are issues with the automatic publishing workflow.

 * Assumes the raw images have already been processed with NodeODM i.e. that `./orthophoto/odm_orthophoto.original.tif` already exists in the mission folder.
 
 * The folder/data structure must follow the guidelines [here](https://seabee-no.github.io/documentation/data-upload.html#sec-data-structure).

In [2]:
# List of mission folders to process
mission_fold_list = [
    r"/home/notebook/shared-seabee-ns9879k/seabirds/2023/agder_lilleslettingen_20230525",
]

temp_dir = r"/home/notebook/cogs/"

In [3]:
# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)

In [4]:
for mission_fold in mission_fold_list:
    mission_fold = os.path.normpath(mission_fold)
    mission_name = os.path.basename(mission_fold)
    print(f"\n################\nProcessing: {mission_name}")
    group, area, date = sb.ortho.parse_mission_data(mission_name)
    config_data = sb.ortho.parse_config(mission_fold)

    print("Preparing orthophoto for publishing.")

    # Is the Orthophoto from ODM or Pix4D?
    odm_ortho_path = os.path.join(
        mission_fold, "orthophoto", "odm_orthophoto.original.tif"
    )
    if os.path.isfile(odm_ortho_path):
        ortho_path = odm_ortho_path
    else:
        ortho_path = os.path.join(
            mission_fold, "orthophoto", "pix4d_orthophoto.original.tif"
        )

    # Standardise and save locally
    temp_path = os.path.join(temp_dir, mission_name + ".tif")
    sb.geo.standardise_orthophoto(
        ortho_path,
        temp_path,
        red_band=1,
        green_band=2,
        blue_band=3,
        nodata=255,
    )

    # Copy to MinIO and delete local version
    stan_path = os.path.join(mission_fold, "orthophoto", mission_name + ".tif")
    sb.storage.copy_file(temp_path, stan_path, minio_client, overwrite=False)
    os.remove(temp_path)

    print("Uploading to GeoServer.")

    layer_name = sb.geo.upload_raster_to_geoserver(
        stan_path,
        SETTINGS.GEOSERVER_USER,
        SETTINGS.GEOSERVER_PASSWORD,
        workspace="geonode",
    )

    print("Publishing to GeoNode.")

    sb.geo.publish_to_geonode(
        layer_name,
        SETTINGS.GEONODE_USER,
        SETTINGS.GEONODE_PASSWORD,
        workspace="geonode",
    )

    print("Updating metadata.")

    abstract = sb.geo.get_html_abstract(mission_fold)
    metadata = {
        "abstract": abstract,
        "date": date.isoformat(),
        "date_type": "creation",
        "attribution": "SeaBee",
    }
    sb.geo.update_geonode_metadata(
        layer_name,
        SETTINGS.GEONODE_USER,
        SETTINGS.GEONODE_PASSWORD,
        metadata,
    )

    print(f"Finished.\n")


################
Processing: agder_lilleslettingen_20230525
Uploading to GeoServer.
Publishing to GeoNode.
Updating metadata.
Finished.

